### Fatorizing the user game matrix into two matrix. These matrixes will represent users and games profils

In [43]:
import numpy as np
import pandas as pd
from reco_systems.filter import filter_df
from reco_systems.user_game_matrix import *
from reco_systems.CF_knn import *
import seaborn as sns
from reco_systems.evaluation import *
from scipy.sparse import csr_matrix
from scipy.linalg import svd
from copy import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# import DB et set min_reviews

folder = "database_cleaned"
avis_clean  = pd.read_csv(f"{folder}/avis_clean.csv", index_col=0)
jeux_clean  = pd.read_csv(f"{folder}/jeux_clean.csv", index_col=0)
users       = pd.read_csv(f"{folder}/users.csv", index_col=0)

min_reviews = 20

In [5]:
# filter data with the minimum reviews
filtered_avis = filter_df(avis_clean, min_reviews)

# create user-game matrix
user_game_ratings, mask_ratings, users_table_assoc, games_table_assoc = get_matrix_user_game(filtered_avis)

11201 10549
3242 3957
3006 3910
3003 3910


In [ ]:
# np.set_printoptions(threshold=np.inf) # see the array without truncation, to truncate, threshold = 0
user_game_ratings.todense()[0].nonzero()[0].size


53

In [ ]:
# transforming the matrix as array for svd
mat_ug = user_game_ratings.toarray()
mat_ug[0].nonzero()[0].shape

In [ ]:
# creating svd matrix

#a, d, b = svd(mat_ug)
a, d, b = np.load("generated_data/a.npy"), np.load("generated_data/d.npy"), np.load("generated_data/b.npy") 
# d contains the diagonal values of a matrix with only diagonal as non nul values

In [ ]:
e = np.dot(a[0].shape, d[0])
b.shape[0]

(3910, (1,))

In [21]:
a.shape, d.shape, b.shape

((3003, 3003), (3003,), (3910, 3910))

In [32]:
diag = np.zeros((a.shape[0], b.shape[0]))
np.fill_diagonal(diag, d)